<h1>Scrape Resumes</h1>
<h2>Be careful not to scrape too much</h2>
The resumes on Indeed are fairly well structured in their html. I want to scrape the job titles and see what signal I have to work with for measuring how often job transitions have occurred.

In [1]:
#Import libraries
import urllib
from bs4 import BeautifulSoup
from selenium  import webdriver
import csv
import datetime
import re
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
import time


def get_soup(start=0):
    '''
    This function loads the resumes database at Indeed.com for new york starting at item number start
    It then scrapes the page and extracts the hyperlinks to the individual resumes.
    It returns a list of all of the hyperlinks associated with a resume.
    '''
    driver = webdriver.Chrome('/Users/christopherluciuk/Downloads/chromedriver')
    #The last concatenation lets us navigate through pages
    driver.get('https://www.indeed.com/resumes?l=new+york&co=US&start='+str(start))
    soup = BeautifulSoup(driver.page_source,"lxml")
    driver.close()
    links = []
    for item in soup.find_all(name='li',attrs={'data-tn-component':'resume-search-result'}):
        for tag in item.find_all(name='a',attrs={'data-tn-element':'resume-result-link[]'}):
            if tag['href'] is None:
                pass
            else:
                links.append('https://www.indeed.com'+str(tag['href']))
    return links

In [2]:
def get_job_pairs(links):
    '''
    This function follows a link and returns a list of the pairs of jobs that appear adjacent
    to each other in time. It returns all the job pairs for all of the links in a list.
    '''
    list_pairs = []
    for url in links:
        driver = webdriver.Chrome('/Users/christopherluciuk/Downloads/chromedriver')
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,"lxml")
        driver.close()
        work_exp = []
        for item in soup.find_all(name='div',attrs={'class':'work-experience-section'}):
            for tag in item.find_all(name='p',attrs={'class':'work_title title'}):
                if tag is None:
                    pass
                else:
                    work_exp.append(tag.text.strip())
        #Now find the chronological pairs
        for i in range(len(work_exp)-1):
            pair = [work_exp[i],work_exp[i+1]]
            list_pairs.append(pair)
        time.sleep(120) #Timer to limit how much I scrape
    return list_pairs

In [ ]:
links = get_soup(start=100)
data = get_job_pairs(links)

In [ ]:
print(data)

In [ ]:
#Write the job pairs to a postgreSQL database

#Create or load database
# Define a database name
# Set your postgres username
dbname = 'indeed_resumes_v1'
username = 'christopherluciuk' # change this to your username
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

#Create
#con = psycopg2.connect(database = dbname, user = username)
#cur = con.cursor()
#Comment out after having created the table
#cur.execute("CREATE TABLE data_table(id INTEGER PRIMARY KEY, Title1 VARCHAR(500), Title2 VARCHAR(500))")

#con.commit()
#Close
#con.close()

con = None
con = psycopg2.connect(database = dbname, user = username)
cur = con.cursor()


#How to add python array to database
sql_query = '''SELECT * FROM data_table ORDER BY ID DESC LIMIT 1;'''
data_out = pd.read_sql_query(sql_query,con)
last_val = int(data_out['id'].values)

for i in range(len(data)):
    item_index = i + (last_val+1) #Need to add this criteria when the database exists!
    data[i].insert(0,item_index)
    cur.execute("INSERT INTO data_table VALUES(%s, %s, %s)", (data[i]))
    
con.commit()
cur.close()
con.close()

In [ ]:
#Check that I wrote the pairs I thought I did
con = None
con = psycopg2.connect(database = dbname, user = username)
sql_query = '''SELECT * FROM data_table;'''
data_out = pd.read_sql_query(sql_query,con)
con.close()
display(data_out)